In [17]:
# Naive Bayes implementation only on 2018's data

# Importing necessary dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from mixed_naive_bayes import MixedNB
from sklearn.preprocessing import LabelEncoder



In [28]:
# DATA PREPROCESSING
trainingData = pd.read_csv("../Data/FIFAWorldCup2018.csv")
print("Dimensions of FIFA WC 2018: {}rows X {}columns".format(trainingData.shape[0],trainingData.shape[1]))

# creating instance of labelencoder
labelencoder = LabelEncoder()
trainingData["home_team_name"] = labelencoder.fit_transform(trainingData["home_team_name"])
trainingData["away_team_name"] = labelencoder.fit_transform(trainingData["away_team_name"])

print(trainingData["home_team_name"])
print(trainingData["away_team_name"])


namesOfColumnsToBeDropped = ["date_GMT","status","attendance","referee","stadium_name", "team_a_xg", "team_b_xg", "btts_percentage_pre_match", 
"over_15_percentage_pre_match", "over_25_percentage_pre_match","over_35_percentage_pre_match", "over_45_percentage_pre_match",
"over_15_HT_FHG_percentage_pre_match", "over_05_HT_FHG_percentage_pre_match", "over_15_2HG_percentage_pre_match", "over_05_2HG_percentage_pre_match","timestamp","Game Week","home_team_goal_timings","away_team_goal_timings",'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg']

trainingData = trainingData.drop(namesOfColumnsToBeDropped,axis=1)

print("Dimensions of FIFA WC 2018 after dropping attributes: {}rows X {}columns".format(trainingData.shape[0],trainingData.shape[1]))

winner = []

#Generating prediction classes W,D,L where W = Win, D = Draw, L = Loss
for index,match in trainingData.iterrows():
    if(match["home_team_goal_count"]>match["away_team_goal_count"]):
        #print("{} {}:{} {}\t Home team won".format(match["home_team_name"],match["home_team_goal_count"],match["away_team_goal_count"],match["away_team_name"]))
        # Adding winner attribute to get result of team1
        #winner.append("{}".format(match["home_team_name"]))
        winner.append(match["home_team_name"])
    
    if(match["home_team_goal_count"]<match["away_team_goal_count"]):
        #print("{} {}:{} {}\t Away team won".format(match["home_team_name"],match["home_team_goal_count"],match["away_team_goal_count"],match["away_team_name"]))
        # Adding winner attribute to get result of team1
        #winner.append("{}".format(match["away_team_name"]))
        winner.append(match["away_team_name"])

    if(match["home_team_goal_count"]==match["away_team_goal_count"]):
        #print("{} {}:{} {}\t Draw".format(match["home_team_name"],match["home_team_goal_count"],match["away_team_goal_count"],match["away_team_name"]))
        # Adding home_team_result attribute to get result of team1
        if(match["home_team_shots"]>=match["away_team_shots"]):
            winner.append(match["home_team_name"])   
        if(match["home_team_shots"]<match["away_team_shots"]):
            winner.append(match["away_team_name"])

            
#trainingData = trainingData.drop(["home_team_name","away_team_name"],axis=1)

trainingData["winner"] = winner

#Writing Dataset.csv
trainingData.to_csv('../Data/Dataset2.csv',index=False)
print(trainingData)
print(trainingData.describe())

Dimensions of FIFA WC 2018: 64rows X 64columns
0     22
1      8
2     16
3     21
4     10
      ..
59    22
60    10
61     6
62     2
63    10
Name: home_team_name, Length: 64, dtype: int64
0     23
1     31
2     13
3     27
4      1
      ..
59     6
60     2
61     9
62     9
63     6
Name: away_team_name, Length: 64, dtype: int64
Dimensions of FIFA WC 2018 after dropping attributes: 64rows X 40columns
    home_team_name  away_team_name  home_team_goal_count  \
0               22              23                     5   
1                8              31                     0   
2               16              13                     0   
3               21              27                     3   
4               10               1                     2   
..             ...             ...                   ...   
59              22               6                     2   
60              10               2                     1   
61               6               9              

In [24]:
# TRAINING AND TESTING DATA PREPARATION
targets = trainingData["winner"]
features = trainingData.drop("winner",axis=1).values

featuresTrain, targetsTrain = features[0:47], targets[0:47]
featuresTest, targetsTest = features[47:], targets[47:]

print(features)


[[22.   23.    5.   ... 10.    2.55  1.44]
 [ 8.   31.    0.   ...  7.5   2.2   1.59]
 [16.   13.    0.   ... 15.    2.25  1.56]
 ...
 [ 6.    9.    2.   ... 15.    2.05  1.69]
 [ 2.    9.    2.   ...  6.5   1.56  2.3 ]
 [10.    6.    4.   ... 15.    2.25  1.57]]


In [25]:
print(targetsTrain)
# DEFINE GAUSSIAN MODEL, TRAIN AND TEST
#model = GaussianNB()
model = MixedNB(categorical_features=[0,1])

#Training model
model.fit(featuresTrain,targetsTrain)

#Predicted and actual values
print(model.classes_)
print(targetsTest)
print(model.predict(featuresTest))
model.score(featuresTest,targetsTest)


0     22.0
1     31.0
2     13.0
3     27.0
4     10.0
5      0.0
6      7.0
7      6.0
8     25.0
9     15.0
10     3.0
11    28.0
12     2.0
13     9.0
14    14.0
15    24.0
16    22.0
17    21.0
18    31.0
19    27.0
20     7.0
21    10.0
22     6.0
23     3.0
24    17.0
25    29.0
26     2.0
27    15.0
28    11.0
29     9.0
30    24.0
31     4.0
32    31.0
33    23.0
34    21.0
35    27.0
36    10.0
37    19.0
38     0.0
39     6.0
40    26.0
41    28.0
42     3.0
43     5.0
44    20.0
45     4.0
46     2.0
Name: winner, dtype: float64


ValueError: Expected y to have classes [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25] but got [ 0.  2.  3.  4.  5.  6.  7.  9. 10. 11. 13. 14. 15. 17. 19. 20. 21. 22.
 23. 24. 25. 26. 27. 28. 29. 31.] instead. Encode your data using sklearn's LabelEncoder.